In [ ]:
import pandas as pd

import glob

from google.oauth2 import service_account
from transform_co_returns import set_dtypes_on
from load_to_gcp import create_bq_schema
# from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
voter_history_directory = r'C:\Users\thewh\Documents\Master Voting History'

# Google Service Credential JSON <- You'll need your own!
# gs_credentials = ServiceAccountCredentials.from_json_keyfile_name('cpc-datawarehouse-51210-cbbf1375a84e.json')
# Establish BigQuery credentials
bq_credentials = service_account.Credentials.from_service_account_file(
    'cpc-datawarehouse-51210-cbbf1375a84e.json', scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

bq_project_id = 'cpc-datawarehouse-51210'
bq_table_id = 'cpc-datawarehouse-51210.co_voterfile.vote-history'

history_integer_col_lst = [
    'VOTER_ID'
]

In [ ]:
# Load vote history data frame
print("Beginning to assemble vote history file.")

allfiles = glob.glob(voter_history_directory + "/*.txt")
history_df = pd.DataFrame()
_lst = []
for _file in allfiles:
    if 'Voting_History' in _file:
        print(f"Loading {_file}")
        _df = pd.read_csv(_file, sep=',', encoding='cp437', index_col=None, header=0, low_memory=False)
        _lst.append(_df)

history_df = pd.concat(_lst)
history_df.reset_index(drop=True, inplace=True)
print("Finished assembling vote history file.")

In [ ]:
history_df = set_dtypes_on(history_df, history_integer_col_lst)

In [ ]:
bq_schema_lst = create_bq_schema(history_df, history_integer_col_lst)
history_df.to_gbq(destination_table=bq_table_id, project_id=bq_project_id, if_exists='replace', table_schema=bq_schema_lst, credentials=bq_credentials)

In [ ]:
del history_df